In [1]:
!pip install datasets transformers huggingface_hub

In [2]:
!pip install --upgrade accelerate>=0.26.0


zsh:1: 0.26.0 not found


In [3]:
import os
import torch
import warnings
from typing import Dict, Any, Union, List

from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    DefaultDataCollator, #
    BatchEncoding
)
from huggingface_hub import login, HfFolder


In [19]:
warnings.filterwarnings("ignore", category=FutureWarning)

HF_USERNAME = "DipeshChaudhary"
MODEL_NAME = "IRIIS-RESEARCH/RoBERTa_Nepali_125M"
DATASET_NAME = f"{HF_USERNAME}/nepali-gector-mlm-guesser-dataset" 
OUTPUT_DIR = "nepali-mlm-guesser-finetuned-model-1"
HUB_MODEL_ID = f"{HF_USERNAME}/{OUTPUT_DIR}"

In [5]:
FP16_MODE = 'bf16' 
PER_DEVICE_BATCH_SIZE = 512 
GRADIENT_ACCUMULATION = 1
LEARNING_RATE = 5e-5
NUM_EPOCHS = 3
MAX_STEPS = -1 # Run for all epochs


In [ ]:
# HF_TOKEN = ""

In [8]:
MAX_SEQ_LENGTH = 128


login(token=HF_TOKEN)
print("✅ Hugging Face login successful.")

✅ Hugging Face login successful.


In [9]:

print("\n" + "="*50)
print(f"LOADING MODEL: {MODEL_NAME}")
print("="*50)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)

# Load the prepared MLM dataset (containing 'input_ids' and 'labels')
print(f"LOADING DATASET: {DATASET_NAME}")
try:
    tokenized_datasets = load_dataset(DATASET_NAME)
except Exception as e:
    print(f"❌ ERROR loading dataset. Ensure {DATASET_NAME} is uploaded to the Hub.")
    print(e)
    tokenized_datasets = None

if not tokenized_datasets:
    raise FileNotFoundError(f"Could not load dataset {DATASET_NAME}. Please check the repo ID.")



LOADING MODEL: IRIIS-RESEARCH/RoBERTa_Nepali_125M
LOADING DATASET: DipeshChaudhary/nepali-gector-mlm-guesser-dataset


In [10]:
tokenized_datasets = tokenized_datasets.rename_column("mlm_input_ids", "input_ids")
tokenized_datasets = tokenized_datasets.rename_column("mlm_labels", "labels")

columns_to_remove = [col for col in tokenized_datasets['train'].column_names if col not in ['input_ids', 'labels', 'attention_mask']]
if columns_to_remove:
    # Need to handle potential KeyError if attention_mask is already gone
    try:
        tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
        print(f"Removed non-essential columns: {columns_to_remove}")
    except KeyError:
        # If the column wasn't there to begin with, ignore the error
        pass

Removed non-essential columns: ['incorrect_sentence', 'correct_sentence', 'mlm_tag_count']


In [11]:
def create_attention_mask(examples):
    """
    Generates a full attention mask (all 1s) for non-padded tokens.
    This is necessary because the input dataset is missing this column.
    """
    # Create an attention mask of 1s for the length of the existing input_ids
    examples['attention_mask'] = [1] * len(examples['input_ids'])
    return examples

# Check if attention_mask is missing before generating it
if 'attention_mask' not in tokenized_datasets['train'].column_names:
    print("\nGenerating missing 'attention_mask' column...")
    # Apply the function across all splits
    tokenized_datasets = tokenized_datasets.map(
        create_attention_mask,
        num_proc=os.cpu_count(),
        desc="Generating attention_mask"
    )
else:
    print("\n'attention_mask' column already exists.")


Generating missing 'attention_mask' column...


In [12]:
def pad_data(examples):
    """
    Pads the input_ids, attention_mask, and labels to the fixed MAX_SEQ_LENGTH (128)
    and sets the label padding tokens to -100.
    """
    
    # Calculate length of the current sequence
    current_length = len(examples['input_ids'])
    
    # Calculate how much padding is needed
    padding_needed = MAX_SEQ_LENGTH - current_length

    if padding_needed > 0:
        # 1. Pad input_ids (using the tokenizer's padding ID)
        examples['input_ids'] = examples['input_ids'] + [tokenizer.pad_token_id] * padding_needed
        
        # 2. Pad attention_mask (using 0s, since padded tokens should not be attended to)
        examples['attention_mask'] = examples['attention_mask'] + [0] * padding_needed
        
        # 3. Pad labels (using -100, so the loss function ignores them)
        examples['labels'] = examples['labels'] + [-100] * padding_needed
    
    # IMPORTANT: Truncate any sequences longer than MAX_SEQ_LENGTH
    # This is a safety measure.
    if current_length > MAX_SEQ_LENGTH:
        examples['input_ids'] = examples['input_ids'][:MAX_SEQ_LENGTH]
        examples['attention_mask'] = examples['attention_mask'][:MAX_SEQ_LENGTH]
        examples['labels'] = examples['labels'][:MAX_SEQ_LENGTH]

    return examples

print(f"\nApplying fixed-length padding (MAX_SEQ_LENGTH={MAX_SEQ_LENGTH}) to dataset...")



Applying fixed-length padding (MAX_SEQ_LENGTH=128) to dataset...


In [13]:
tokenized_datasets = tokenized_datasets.map(
    pad_data,
    num_proc=os.cpu_count(), # Use all available cores for fast processing
    desc=f"Padding to fixed length {MAX_SEQ_LENGTH}"
)


data_collator = DefaultDataCollator(return_tensors="pt") 

print(f"\nCRITICAL FIX: Switched to DefaultDataCollator to preserve pre-set labels.")

# Let's verify the feature names in the dataset:
print("\nDataset features (MUST contain 'input_ids', 'labels', and 'attention_mask'):")
print(tokenized_datasets['train'].column_names)

# --- 6. TRAINING ARGUMENTS ---
print("\n" + "="*50)
print("TRAINING ARGUMENTS (H100 OPTIMIZED)")
print("="*50)



CRITICAL FIX: Switched to DefaultDataCollator to preserve pre-set labels.

Dataset features (MUST contain 'input_ids', 'labels', and 'attention_mask'):
['input_ids', 'labels', 'attention_mask']

TRAINING ARGUMENTS (H100 OPTIMIZED)


In [20]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    max_steps=MAX_STEPS, 
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE * 2, 
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    
    bf16=(FP16_MODE == 'bf16'), 
    # fp16=(FP16_MODE == 'fp16')
    
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    optim="adamw_torch",
    
    # Checkpointing and Logging
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    load_best_model_at_end=False, # We usually want the last checkpoint for finetuning
    save_total_limit=1, # Save only the last checkpoint
    
    # Uploading to Hub
    push_to_hub=True,
    hub_model_id=HUB_MODEL_ID,
    hub_token=HF_TOKEN,
    hub_private_repo=True,
    
    # CPU usage - utilize all available cores
    dataloader_num_workers=os.cpu_count(),
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator 
)


In [23]:

print("\n" + "="*50)
print("STARTING MLM FINE-TUNING")
print(f"Effective Batch Size: {PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION}")
print(f"Precision: {FP16_MODE.upper()}")
print(f"Forced Max Sequence Length: {MAX_SEQ_LENGTH}")
print("="*50)
train_result = trainer.train()
print("\n🎉 TRAINING COMPLETE!")
print("\n🎉 TRAINING COMPLETE!")

print("\n" + "="*50)
print("FINE-TUNING SCRIPT EXECUTION FINISHED.")
print("="*50)


STARTING MLM FINE-TUNING
Effective Batch Size: 512
Precision: BF16
Forced Max Sequence Length: 128


Epoch,Training Loss,Validation Loss
1,0.902500,0.754820
2,0.535000,0.482485
3,0.389300,0.428481



🎉 TRAINING COMPLETE!

🎉 TRAINING COMPLETE!

FINE-TUNING SCRIPT EXECUTION FINISHED.


In [24]:
print("\n" + "="*50)
print("SAVING AND UPLOADING FINAL MODEL")
print("="*50)

# Save the final model locally
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)


trainer.push_to_hub(commit_message="MLM Guesser fine-tuning completed ")
print(f"Model successfully pushed to the Hub at: {HUB_MODEL_ID}")



SAVING AND UPLOADING FINAL MODEL


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Model successfully pushed to the Hub at: DipeshChaudhary/nepali-mlm-guesser-finetuned-model-1
